# Моя реализация алгоритма логистической регрессии

Импортируем необходимые библиотеки для создания класса и для сравнения результатов предсказания с встроенной в sklearn моделью.

In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

### Данная модель принимает на вход следующие параметры:
n_iter - количество итераций алгоритма. По умолчанию: 100

learning_rate - коэффициент скорости обучения градиентного спуска. По умолчанию: 0.1

metric - метрика, рассчитывающаяся в процессе и в конце обучения модели. Поддерживаемые значения: 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'. По умолчанию: None

reg - тип регуляризации. Поддерживаемые значения: 'l1' - для L1 регуляризации, 'l2' - для L2, 'elasticnet' - для комбинации L1 и L2 регуляризаций. По умолчанию: None

l1_coef - коэффициент регуляризации L1. По умолчанию: 0

l2_coef - коэффициент регуляризации L2. По умолчанию: 0

sgd_sample - размер выборки для обучения стохастического градиентного бустинга. Значения от 0 до 1 задают долю исходной выборки, а целые значения >1 - количество элементов исходной выборки, которое будет использоваться. По умолчанию: None

random_state - зерно генератора для воспроизводимости результатов

### MyLogReg поддерживает следующие методы:
fit(X: pd.DataFrame(), y: pd.Series(), verbose: None) - обучает модель на полученных данных. Доступна встроенная регуляризация трех типов, параллельный расчет одной из пяти обозначенных метрик, вывод лога компиляции и стохастический градиентный спуск для больших исходных данных.

get_coef() - возвращает коэффициенты обученной модели

predict(X) - возвращает предсказанные на основе X значения y

get_best_score() - возвращает заданную при инициализации метрику, рассчитанную для обученной модели

In [115]:
import numpy as np
import pandas as pd
import random

class MyLogReg:
    def __init__(self, n_iter=10, learning_rate=0.1, metric=None, reg=None, l1_coef=0, l2_coef=0, sgd_sample=None, random_state=42):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample = sgd_sample
        self.random_state = random_state
        self.weights = None

    def __str__(self):
        return f'MyLogReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'

    def fit(self, X: pd.DataFrame(), y: pd.Series(), verbose=False):
        np.random.seed(self.random_state)
        random.seed(self.random_state)

        X = np.hstack((np.ones((X.shape[0], 1)), X.values))
        y = y.values
        weights = np.ones(X.shape[1])
        n_full = len(y)

        if self.sgd_sample:
            if self.sgd_sample <= 1.0:
                n = round(n_full * self.sgd_sample)
            else:
                n = self.sgd_sample
        else:
            n = n_full

        for i in range(1, self.n_iter + 1):
            if self.sgd_sample:
                sample_row_idx = random.sample(range(n_full), n)
                X_sample = X[sample_row_idx, :]
                y_sample = y[sample_row_idx]
            else:
                X_sample = X
                y_sample = y

            y_pred = 1 / (1 + np.exp(-np.dot(X_sample, weights)))
            y_pred_full = 1 / (1 + np.exp(-np.dot(X, weights)))

            if self.reg == 'l1':
                log_loss = -np.mean(y * np.log(y_pred_full + 1e-15) + (1 - y) * np.log(1 - y_pred_full + 1e-15)) + self.l1_coef * np.sum(np.abs(weights))
                gradient = np.dot(X_sample.T, (y_pred - y_sample)) / n + self.l1_coef * np.sign(weights)
            elif self.reg == 'l2':
                log_loss = -np.mean(y * np.log(y_pred_full + 1e-15) + (1 - y) * np.log(1 - y_pred_full + 1e-15)) + self.l2_coef * np.sum(weights ** 2)
                gradient = np.dot(X_sample.T, (y_pred - y_sample)) / n + 2 * self.l2_coef * weights
            elif self.reg == 'elasticnet':
                log_loss = -np.mean(y * np.log(y_pred_full + 1e-15) + (1 - y) * np.log(1 - y_pred_full + 1e-15)) + self.l1_coef * np.sum(np.abs(weights)) + self.l2_coef * np.sum(weights ** 2)
                gradient = np.dot(X_sample.T, (y_pred - y_sample)) / n + self.l1_coef * np.sign(weights) + 2 * self.l2_coef * weights
            else:
                log_loss = -np.mean(y * np.log(y_pred_full + 1e-15) + (1 - y) * np.log(1 - y_pred_full + 1e-15))
                gradient = np.dot(X_sample.T, (y_pred - y_sample)) / n

            if callable(self.learning_rate):
                learning_rate = self.learning_rate(i)
            else:
                learning_rate = self.learning_rate

            weights -= learning_rate * gradient

            if verbose or i == self.n_iter:
                y_pred_full = 1 / (1 + np.exp(-np.dot(X, weights)))
                y_classes = (y_pred_full >= 0.5).astype(int)

                if self.metric == 'accuracy':
                    metric = np.mean(y_classes == y)
                elif self.metric == 'precision':
                    metric = np.sum((y_classes == 1) & (y_classes == y)) / np.sum(y_classes == 1)
                elif self.metric == 'recall':
                    metric = np.sum((y == 1) & (y_classes == y)) / np.sum(y == 1)
                elif self.metric == 'f1':
                    precision = np.sum((y_classes == 1) & (y_classes == y)) / np.sum(y_classes == 1)
                    recall = np.sum((y == 1) & (y_classes == y)) / np.sum(y == 1)
                    metric = 2 * precision * recall / (precision + recall)
                elif self.metric == 'roc_auc':
                    if np.sum(y == 1) == 0 or np.sum(y == 0) == 0:
                        metric = 'ValueError'
                    else:
                        y_pred_sort = y_pred_full.argsort()[::-1]
                        y_sort = y[y_pred_sort]
                        metric = 1/np.sum(y==1)/np.sum(y==0) * np.sum([np.sum((y_sort[i]<y_sort).astype(int)*(0*(y_pred_sort[i]>y_pred_sort).astype(int) + 0.5*((y_pred_sort[i]==y_pred_sort).astype(int)) + 1*(y_pred_sort[i]<y_pred_sort).astype(int))) for i in range(len(y))])

                if verbose:
                    if self.metric:
                        if i == 1:
                            print(f'start | loss: {log_loss} | {self.metric}: {metric}')
                        elif i % verbose == 0:
                            print(f'{i} | loss: {log_loss} | {self.metric}: {metric}')
                    else:
                        if i == 1:
                            print(f'start | loss: {log_loss}')
                        elif i % verbose == 0:
                            print(f'{i} | loss: {log_loss}')

                if i == self.n_iter and metric is not None:
                    self.metric_value = metric

        self.weights = weights

    def get_coef(self):
        return self.weights[1:]

    def predict_proba(self, X):
        X = np.hstack((np.ones((X.shape[0], 1)), X.values))
        return 1 / (1 + np.exp(-np.dot(X, self.weights)))

    def predict(self, X):
        X = np.hstack((np.ones((X.shape[0], 1)), X.values))
        return (1 / (1 + np.exp(-np.dot(X, self.weights))) > 0.5).astype(int)

    def get_best_score(self):
        return self.metric_value

Создаем выборку размера 10000 с помощью встроенной в sklearn функции

In [80]:
X, y = make_classification(n_samples=10000, n_features=14, n_informative=10, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)

Обучим нашу модель и спрогнозируем классы для тех же данных. Затем обучим встроенную в sklearn модель и сравним их метрики и прогнозы.

In [116]:
model = MyLogReg(metric='roc_auc')
model.fit(X, y)
y_pred = model.predict(X)

accuracy_score(y, y_pred), precision_score(y, y_pred), recall_score(y, y_pred), f1_score(y, y_pred), roc_auc_score(y, y_pred)

(0.5284, 0.5240677966101694, 0.6184, 0.5673394495412845, 0.5284)

In [117]:
model_sk = LogisticRegression()
model_sk.fit(X, y)
y_pred_sk = model.predict(X)

accuracy_score(y, y_pred_sk), precision_score(y, y_pred_sk), recall_score(y, y_pred_sk), f1_score(y, y_pred_sk), roc_auc_score(y, y_pred_sk)

(0.5284, 0.5240677966101694, 0.6184, 0.5673394495412845, 0.5284)

In [114]:
(y_pred==y_pred_sk).sum()

10000

Как мы видим из результатов сравнения предсказаний, построенная модель дает тот же результат, что и существующая в библиотеке sklearn. Помимо прочего, данная реализация поддерживает расчет метрик в процессе обучения, регуляризацию и стохастический градиентный спуск, из чего я делаю вывод, что модель успешна.